## セットアップ

In [1]:
!pip install -q transformers sentencepiece sentence-transformers \
    torch faiss-cpu datasets scipy scikit-learn numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 105.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 69.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 36.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# VRAMの環境変数設定
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

## モデルの準備

In [3]:
import torch
from transformers import AutoModel, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("pfnet/plamo-embedding-1b", trust_remote_code=True)
model = AutoModel.from_pretrained("pfnet/plamo-embedding-1b", trust_remote_code=True)

device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)
model.eval()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.30k [00:00<?, ?B/s]

tokenization_plamo.py:   0%|          | 0.00/7.63k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/pfnet/plamo-embedding-1b:
- tokenization_plamo.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


tokenizer.model:   0%|          | 0.00/805k [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/1.09k [00:00<?, ?B/s]

modeling_plamo.py:   0%|          | 0.00/38.9k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/pfnet/plamo-embedding-1b:
- modeling_plamo.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors:   0%|          | 0.00/2.10G [00:00<?, ?B/s]

PlamoBiModel(
  (embed_tokens): Embedding(50112, 2048, padding_idx=3)
  (layers): ModifiedPlamoDecoder(
    (layers): ModuleList(
      (0-15): 16 x ModifiedPlamoDecoderLayer(
        (self_attn): ModifiedAttention(
          (qkv_proj): Linear(in_features=2048, out_features=2304, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): RotaryEmbedding()
        )
        (mlp): DenseMLP(
          (gate_up_proj): Linear(in_features=2048, out_features=16384, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
        )
        (norm): RMSNorm()
        (norm2): RMSNorm()
      )
    )
  )
  (norm): RMSNorm()
)

## 埋め込み計算モジュールの定義

In [4]:
import numpy as np

def embed_texts(
    texts: list[str],
    batch_size: int = 32,
    is_query: bool = False,
) -> np.ndarray:
    """
    texts           : 埋め込み対象の文字列リスト
    batch_size      : 一度に投入する文の数（デフォルト：32）
    is_query        : True の場合は model.encode_query を使用
                                False の場合は model.encode_document を使用
    """
    all_embs = []

    with torch.inference_mode():
      # 自動混合精度演算
      with torch.amp.autocast("cuda"):
            for i in range(0, len(texts), batch_size):
                batch = texts[i : i + batch_size]
                if is_query:
                    embs = model.encode_query(
                        batch,
                        tokenizer,
                    )
                else:
                    embs = model.encode_document(
                        batch,
                        tokenizer,
                    )

                # embs は torch.Tensor
                embs_np = embs.to("cpu").numpy()  # NumPy に変換
                # VRAM を開放する
                del embs
                torch.cuda.empty_cache()
                embs_np = np.nan_to_num(embs_np)
                all_embs.append(embs_np)

    return np.vstack(all_embs)


## JSTSによるモデルの評価

In [ ]:
from datasets import load_dataset

ds = load_dataset("sbintuitions/JMTEB", "jsts")
jsts = ds["test"]
print(jsts.column_names)  # ['sentence_pair_id', 'yjcaptions_id', 'sentence1', 'sentence2', 'label']

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# JSTS の全ペアを「文1／文2」と「スコア」に分解
sent1 = jsts["sentence1"]
sent2 = jsts["sentence2"]
gold_score = np.array(jsts["label"], dtype=float)  # 0〜5 の連続値

# 埋め込み
emb1 = embed_texts(sent1)
emb2 = embed_texts(sent2)

# コサイン類似度
# N×2048 の行列同士のコサイン類似度を一度に出す
sim_matrix = cosine_similarity(emb1, emb2)
# 対角成分だけを取り出せばペアごとの類似度が得られる
cos_sim = np.diag(sim_matrix)  # shape=(N,)

In [ ]:
# スピアマンとピアソンの相関係数を計算
from scipy.stats import spearmanr, pearsonr

spearman_corr, _ = spearmanr(cos_sim, gold_score)
pearson_corr, _  = pearsonr(cos_sim, gold_score)

print(f"Spearman: {spearman_corr:.4f}")
print(f"Pearson : {pearson_corr:.4f}")

## インデックス用データ読み込み・前処理・インデックス化

In [5]:
from google.colab import drive
drive.mount('/content/drive')
# drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


### インデックス化: livedoor-news-corpus

In [ ]:
from datasets import load_dataset
# Livedoor ニュースコーパス (約7,300記事) を train/val/test に分割読み込み
ds = load_dataset(
    "shunk031/livedoor-news-corpus",
    train_ratio=0.8,
    val_ratio=0.1,
    test_ratio=0.1,
    shuffle=False,
)
train = ds["train"]
print(train)

In [ ]:
# 検索対象：記事のタイトル＋本文
corpus_texts = [
    f"{row['title']}。{row['content']}"
    for row in train
]

# （例）最初の5件をクエリとして流用
query_texts = corpus_texts[:5]
# 正解ID (今回は自身の記事をトップ1に返す recall@1)
true_ids = list(range(len(query_texts))) # true_ids = [0, 1, 2, ..., 5]

In [ ]:
# 記事のタイトルと本文(corpus_texts)をjsonl形式で保存

import json

with open('/content/drive/MyDrive/Colab Notebooks/data/text_embedding_search/corpus.jsonl', 'w', encoding='utf-8') as f:
    for text in corpus_texts:
        json.dump({'text': text}, f, ensure_ascii=False)
        f.write('\n')


In [ ]:
# 文を埋め込み化する（時間のかかる処理）
xb = embed_texts(corpus_texts, batch_size=8, is_query=False)  # index 用
xq = embed_texts(query_texts,  batch_size=8, is_query=True)   # query 用

In [ ]:
import faiss

d = xb.shape[1]  # 埋め込み次元
M = 32           # 各点の近傍リンク数
efC = 200        # 構築時の探索深さ

index = faiss.IndexHNSWFlat(d, M)
index.hnsw.efConstruction = efC

# 埋め込みベクトルを追加
index.add(xb)

In [ ]:
index.hnsw.efSearch = 50  # 検索時の探索深さ
k = 5                    # top-k 件取得
D, I = index.search(xq, k) # 検索を実行

In [ ]:
recall1 = np.mean([1 if true_ids[i] in I[i,:1] else 0 for i in range(len(true_ids))])
print(f"Recall@1: {recall1:.3f}")

In [ ]:
# 埋め込みと Faiss インデックスを保存
np.save('/content/drive/MyDrive/Colab Notebooks/data/text_embedding_search/embeddings_xb.npy', xb)
faiss.write_index(index, '/content/drive/MyDrive/Colab Notebooks/data/text_embedding_search/hnsw_index.faiss')

In [ ]:
for i in range(len(query_texts)):
    print(f"Query: {query_texts[i]}")
    for rank, idx in enumerate(I[i]):
        print(f"Top {rank+1}: {corpus_texts[idx]}")
    print("========")

### インデックス化: SFC研究会データ

In [ ]:
# スクレイピング済みデータを取得
! git clone https://github.com/kota-yata/Syllabus-proto.git

Cloning into 'Syllabus-proto'...
remote: Enumerating objects: 729, done.
remote: Counting objects: 100% (249/249), done.
remote: Compressing objects: 100% (136/136), done.
remote: Total 729 (delta 149), reused 180 (delta 95), pack-reused 480 (from 1)
Receiving objects: 100% (729/729), 4.47 MiB | 16.53 MiB/s, done.
Resolving deltas: 100% (401/401), done.


In [ ]:
import json

filepath = '/content/Syllabus-proto/assets/result-2025s.json'
with open(filepath, 'r', encoding='utf-8') as f:
    all_class_data = json.load(f)

In [ ]:
lab_corpus_texts = [] # 埋め込みモデルに入力するテキストのリスト
lab_corpus_metadata = [] # メタデータを持つリスト

# target_subjects の "A", "B" は全角
target_subjects = {"研究会Ａ", "研究会Ｂ"}

for idx, item in enumerate(all_class_data):
    # --- 必須情報の取得 ---
    sort_id = item.get('sort_id')
    subject_name = item.get('subject_name', '').strip() # 空の場合や前後の空白に対応
    about_text = item.get('about', '').strip()
    lang = item.get('lang', '').strip()

    # --- モデル非対応のため外国語は除外 ---
    if lang != '日本語':
        continue

    # --- subject_name がターゲットに含まれているかチェック ---
    if subject_name not in target_subjects:
        continue

    # --- ID, subject_name, about がないと意味がないのでスキップ ---
    if not sort_id or not subject_name or not about_text:
        print(f"Skipping item due to missing essential fields: {sort_id or 'Unknown ID'}")
        continue

    # --- 検索対象テキストの生成 ---
    # 研究会名と概要文を結合
    staff_names = [staff.get('staff_name', '') for staff in item.get('staffs', [])]
    staffs = ",".join(filter(None, staff_names))
    text_for_embedding = f"{subject_name}: {staffs}\n{about_text}"
    lab_corpus_texts.append(text_for_embedding)

    # --- メタデータの抽出・整形 ---
    url = item.get('url', '')
    # fields から学部・分野を取得 (リスト形式なので最初のものを取得する例)
    # fields が空リストの場合や、リスト内の辞書が空の場合のエラーを防ぐ
    faculty = ''
    field = ''
    if item.get('fields'): # fields リストが存在するか確認
        first_field = item['fields'][0] if item['fields'] else {} # リストが空でないか確認
        faculty = first_field.get('faculty', '')
        field = first_field.get('field', '')

    term = item.get('term', '')

    metadata = {
        'sort_id': sort_id,
        'subject_name': subject_name,
        'about': about_text, # スニペットは検索時に生成するので、ここでは全文保持
        'url': url,
        'staffs': ", ".join(filter(None, staff_names)), # 教員名をカンマ区切り文字列に
        'faculty': faculty,
        'field': field,
        'term': term
    }
    lab_corpus_metadata.append(metadata)

    if len(about_text) < 30:
        print(f"Found too short about text: {sort_id}, {staffs}")

print(f"Processed {len(lab_corpus_texts)} items.")
print(f"Total items skipped: {len(all_class_data) - len(lab_corpus_texts)}") # スキップされた数を計算

Found too short about text: X48645, 清水　たくみ
Found too short about text: X37781, 大木　聖子
Found too short about text: X28865, 諏訪　正樹
Found too short about text: X47266, 東海林　祐子
Found too short about text: X47266, 東海林　祐子
Skipping item due to missing essential fields: X44849
Found too short about text: X49435, 福島　康仁
Processed 138 items.
Total items skipped: 612


In [ ]:
# about_textの分析
mean_about_length = 0
longest_about = 0
for item in all_class_data:
    sort_id = item.get('sort_id')
    subject_name = item.get('subject_name', '').strip()
    staff_names = [staff.get('staff_name', '') for staff in item.get('staffs', [])]
    staffs = ",".join(filter(None, staff_names))
    about_text = item.get('about', '').strip()

    if subject_name not in target_subjects:
        continue

    if about_text:
        mean_about_length += len(about_text)

    if len(about_text) < 30:
        print(f"Found about text < 30: {sort_id}, {staffs}")

    if len(about_text) > longest_about:
        longest_about = len(about_text)

mean_about_length /= len(lab_corpus_texts)

print(f"Mean about length: {mean_about_length}")
print(f"Longest about length: {longest_about}")

Found about text < 30: X48645, 清水　たくみ
Found about text < 30: X37781, 大木　聖子
Found about text < 30: X28865, 諏訪　正樹
Found about text < 30: X47266, 東海林　祐子
Found about text < 30: X47266, 東海林　祐子
Found about text < 30: X44849, 藤井　進也
Found about text < 30: X49435, 福島　康仁
Mean about length: 257.5869565217391
Longest about length: 1330


In [ ]:
# lab_corpus_texts をJSONファイルに保存
corpus_texts_filepath = '/content/drive/MyDrive/Colab Notebooks/data/text_embedding_search/lab_corpus_texts.json'
try:
    with open(corpus_texts_filepath, 'w', encoding='utf-8') as f:
        # ensure_ascii=False で日本語がそのまま保存されるようにする
        # indent=4 で整形して見やすくする
        json.dump(lab_corpus_texts, f, ensure_ascii=False, indent=4)
    print(f"Successfully saved corpus texts to {corpus_texts_filepath}")
except IOError as e:
    print(f"Error saving corpus texts: {e}")

# lab_corpus_metadata をJSONファイルに保存
corpus_metadata_filepath = '/content/drive/MyDrive/Colab Notebooks/data/text_embedding_search/lab_corpus_metadata.json'
try:
    with open(corpus_metadata_filepath, 'w', encoding='utf-8') as f:
        json.dump(lab_corpus_metadata, f, ensure_ascii=False, indent=4)
    print(f"Successfully saved corpus metadata to {corpus_metadata_filepath}")
except IOError as e:
    print(f"Error saving corpus metadata: {e}")

Successfully saved corpus texts to /content/drive/MyDrive/Colab Notebooks/data/text_embedding_search/lab_corpus_texts.json
Successfully saved corpus metadata to /content/drive/MyDrive/Colab Notebooks/data/text_embedding_search/lab_corpus_metadata.json


In [ ]:
lab_corpus_texts_emb = embed_texts(lab_corpus_texts, is_query=False)

In [ ]:
# 埋め込みをdriveに保存
np.save('/content/drive/MyDrive/Colab Notebooks/data/text_embedding_search/lab_corpus_texts_emb.npy', lab_corpus_texts_emb)

In [ ]:
import faiss

d = lab_corpus_texts_emb.shape[1]  # 埋め込み次元
M = 32           # 各点の近傍リンク数
efC = 200        # 構築時の探索深さ

lab_index = faiss.IndexHNSWFlat(d, M)
lab_index.hnsw.efConstruction = efC

# 埋め込みベクトルを追加
lab_index.add(lab_corpus_texts_emb)

In [ ]:
faiss.write_index(lab_index, '/content/drive/MyDrive/Colab Notebooks/data/text_embedding_search/lab_hnsw_index.faiss')

## 保存したインデックスを読み込んで使用

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
!pip install faiss-cpu
import faiss

### livedoor-news-corpusを読み込んで検索

In [ ]:
# corpus_textsの読み込み
import json

corpus_texts = []
with open('/content/drive/MyDrive/Colab Notebooks/data/text_embedding_search/corpus.jsonl', 'r', encoding='utf-8') as f:
    for line in f:
        corpus_texts.append(json.loads(line)['text'])

In [ ]:
xb = np.load('/content/drive/MyDrive/Colab Notebooks/data/text_embedding_search/embeddings_xb.npy')
index = faiss.read_index('/content/drive/MyDrive/Colab Notebooks/data/text_embedding_search/hnsw_index.faiss')

In [ ]:
query_texts = ["渋谷の事件についてのニュース", "プロ野球の試合結果", "新作映画のレビュー"]
xq = embed_texts(query_texts, is_query=True)

In [ ]:
index.hnsw.efSearch = 50  # 検索時の探索深さ
k = 5  # 上位k件を取得
D, I = index.search(xq, k)  # Dは距離、Iはインデックス

In [ ]:
for i, q in enumerate(query_texts):
  print(f"\n[Query] {q}")
  for j in range(k):
    doc_id = I[i, j] # i番目のクエリにつき5つのID
    print(f" RANK {j+1}: {corpus_texts[doc_id][:50]}...")

### SFC研究会のデータを読み込んで検索

In [8]:
import json

filepath = '/content/drive/MyDrive/Colab Notebooks/data/text_embedding_search/lab_corpus_texts.json'
with open(filepath, 'r', encoding='utf-8') as f:
    lab_corpus_texts = json.load(f)

filepath = '/content/drive/MyDrive/Colab Notebooks/data/text_embedding_search/lab_corpus_metadata.json'
with open(filepath, 'r', encoding='utf-8') as f:
    lab_corpus_metadata = json.load(f)

In [9]:
lab_index = faiss.read_index('/content/drive/MyDrive/Colab Notebooks/data/text_embedding_search/lab_hnsw_index.faiss')

In [10]:
query_texts = ["**AI（人工知能）**は、機械が学習・推論・認識・判断といった人間のような知的能力を模倣する技術。機械学習、深層学習などの手法を用い、データに基づきパターンを学習し予測・分類を行う。自然言語処理や画像認識、強化学習など応用分野は多岐にわたる。"]
xq = embed_texts(query_texts, is_query=True)

In [11]:
lab_index.hnsw.efSearch = 100  # 検索時の探索深さ
k = 5  # 上位k件を取得
D, I =lab_index.search(xq, k)  # Dは距離、Iはインデックス

for i, q in enumerate(query_texts):
  print(f"\n[Query] {q}")
  for j in range(k):
    doc_id = I[i, j] # i番目のクエリにつき5つのID
    print(f" RANK {j+1}: {lab_corpus_texts[doc_id][:200]}...")



[Query] **AI（人工知能）**は、機械が学習・推論・認識・判断といった人間のような知的能力を模倣する技術。機械学習、深層学習などの手法を用い、データに基づきパターンを学習し予測・分類を行う。自然言語処理や画像認識、強化学習など応用分野は多岐にわたる。
 RANK 1: 研究会Ｂ: 矢作　尚久
データサイエンスの世界では、データが生成されるプロセスを正確に把握し、同時にその深層を想像する能力と高い倫理観を要する。気象・金融・交通・物流・生活における各種センサー等のあらゆるデータは、人間の活動に紐付いている。人間の活動は、全てひとり一人の「脳と身体の状態」によって規定されている。しかしながら、データを扱う者たちが、そのデータが生成される背景、あるいはデータを紐付ける...
 RANK 2: 研究会Ａ: ショウ，　ラジブ
This seminar will focus on different issues of environment, disaster and development in Asia...
 RANK 3: 研究会Ｂ: 華　金玲
近年のインターネット、携帯電話、IoT、5Gなどの情報通信技術の発達により、情報へのアクセスが容易になり、我々の生活の利便性が向上した。そのような情報通信の社会基盤に大きく関わっているのがデジタル政策である。
この研究会では、5GのユースケースやChatGPT、生成AI、メタバース、自動運転などのような新しいテクノロジーの利活用と普及について関連動向を幅広く取り上げ、議論する...
 RANK 4: 研究会Ｂ: 華　金玲
近年のインターネット、携帯電話、IoT、5Gなどの情報通信技術の発達により、情報へのアクセスが容易になり、我々の生活の利便性が向上した。そのような情報通信の社会基盤に大きく関わっているのがデジタル政策である。
この研究会では、5GのユースケースやChatGPT、生成AI、メタバース、自動運転などのような新しいテクノロジーの利活用と普及について関連動向を幅広く取り上げ、議論する...
 RANK 5: 研究会Ａ: 武田　圭史
メディア表現（映像、光、音響等）, 生成AI, UAV(ドローン), VR/AR/XRなど広範な先端技術と表現メディアの応用について取り扱います。


映像メディア
目的：様々

## gradioで検索機能を実装

In [12]:
!pip install -q gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 128.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.8 MB/s eta 0:00:00


### livedoorの記事検索

In [ ]:
import pandas as pd

# 検索用関数の定義
def search(query: str, k: int = 5, length: int = 30):
    # クエリ埋め込み
    q_emb = embed_texts([query], is_query=True)
    # Faiss 検索
    D, I = index.search(q_emb, k)
    # 結果整形
    results = []
    for score, idx in zip(D[0], I[0]):
        title, snippet = corpus_texts[idx].split("。", 1)  # タイトル・本文を分割
        results.append({"title": title, "snippet": snippet[:length]+"…", "score": float(score)})

    df = pd.DataFrame(results) # gradio用にDFに変換

    return df


In [ ]:
search("パソコン", k=5)

In [ ]:
import gradio as gr

iface = gr.Interface(
    fn=search,
    inputs=[
        gr.Textbox(lines=2, placeholder="検索クエリを入力"),
        gr.Slider(minimum=1, maximum=10, step=1, label="Top k 件数")
    ],
    outputs=gr.Dataframe(
        headers=["title", "snippet", "score"],
        row_count=5
    ),
    title="Livedoor ニュース検索デモ",
    description="pfnet/plamo-embedding-1b + Faiss HNSW による類似ニュース検索"
)

iface.launch(share=True)

### SFC研究会の検索

In [38]:
import pandas as pd

# 検索用関数の定義
def lab_search(query: str, k: int = 5, length: int = 50):
    # クエリ埋め込み
    q_emb = embed_texts([query], is_query=True)
    # Faiss 検索
    D, I = lab_index.search(q_emb, k)
    # 結果整形
    results = []
    for score, idx in zip(D[0], I[0]):
        data = lab_corpus_metadata[idx]
        title = data.get('subject_name')
        staffs = data.get('staffs')
        snippet = data.get('about')
        url = data.get('url')
        if len(snippet) > length:
            snippet = snippet[:length]+"…"
        results.append({"title": title, "staffs": staffs, "snippet": snippet, "url": url, "score": float(score)})

    df = pd.DataFrame(results) # gradio用にDFに変換

    return df


In [ ]:
import gradio as gr

iface = gr.Interface(
    fn=lab_search,
    inputs=[
        gr.Textbox(lines=2, placeholder="検索クエリを入力"),
        gr.Slider(minimum=1, maximum=10, step=1, label="Top k 件数")
    ],
    outputs=gr.Dataframe(
        headers=["title", "staffs", "snippet", "url", "score"],
        row_count=5
    ),
    title="【デモ】SFC研究会のベクトル検索",
    description="pfnet/plamo-embedding-1b + Faiss HNSW による類似文検索"
)

iface.launch(share=True)